<a href="https://colab.research.google.com/github/maryamhashemi/Persian_VQA/blob/master/feature_extraction_pvqa_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepdish


In [ ]:
# import os
from zipfile import ZipFile

# #If the downloaded file is a zip file than you can use below function to unzip it.
def unzip(dir,where):
    with ZipFile(dir) as zipf:
        zipf.extractall(where)
    print("File Unzipped!")
# uncompress_features_labels('/content/drive/My Drive/Persian_VQA/mscoco_train2014_english.zip','/content/drive/My Drive/Persian_VQA/')

In [ ]:
import os
DROPOUT_RATE = 0.5
EMBEDDING_DIM = 300
EPOCHS = 10
BATCH_SIZE = 256
SEQ_LENGTH = 100
VOCAB_SIZE = 1000
OOV_TOK = "<OOV>"

BASE_PATH = '/content/drive/My Drive/Persian_VQA/'
QUESTION_TRAIN_PATH =   os.path.join(BASE_PATH, 'OpenEnded_mscoco_train2014_questions.json')
ANNOTATION_TRAIN_PATH = os.path.join(BASE_PATH, 'mscoco_train2014_annotations.json')
# IMAGE_TRAIN_PATH = os.path.join(BASE_PATH, 'train_images_1000')
IMAGE_TRAIN_PATH = os.path.join('/content/', 'train2014')

QUESTION_VAL_PATH =   os.path.join(BASE_PATH, 'OpenEnded_mscoco_val2014_questions.json')
ANNOTATION_VAL_PATH = os.path.join(BASE_PATH, 'mscoco_val2014_annotations.json')
# IMAGE_VAL_PATH = os.path.join(BASE_PATH, 'val_images_500')
IMAGE_VAL_PATH = os.path.join('/content/', 'val2014')

QUESTION_TEST_PATH =   os.path.join(BASE_PATH, '...')
ANNOTATION_TEST_PATH = os.path.join(BASE_PATH, '...')
# IMAGE_TEST_PATH = os.path.join(BASE_PATH, 'val_images_500')
IMAGE_TEST_PATH = os.path.join('/content/', 'test')

In [ ]:
!wget http://images.cocodataset.org/zips/train2014.zip
# !wget http://images.cocodataset.org/zips/val2014.zip
# !wget http://images.cocodataset.org/zips/test2015.zip

--2020-08-12 04:48:09--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.162.35
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.162.35|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  35.0MB/s    in 5m 9s   

2020-08-12 04:53:19 (41.7 MB/s) - ‘train2014.zip’ saved [13510573713/13510573713]



In [ ]:
unzip('/content/train2014.zip','/content/')
# unzip('/content/val2014.zip','/content/')
# unzip('/content/test2015.zip','/content/')

File Unzipped!


In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.models import Model

In [ ]:
from tensorflow.keras.applications.nasnet import NASNetLarge, preprocess_input
from tensorflow.keras.models import Model

In [ ]:
from keras.applications.resnet import ResNet152,preprocess_input 
from tensorflow.keras.models import Model

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', pooling='avg' , include_top=False)


In [ ]:
base_model.layers[-1]

In [ ]:
import tensorflow
import os
import re
import logging
import numpy as np
import pandas as pd
from PIL import Image
import deepdish as dd
# from constants import *
from tensorflow.keras.models import Model
# from tensorflow.keras.applications import VGG19
# from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# from tensorflow.keras.applications import ResNet152
# from tensorflow.keras.applications import preprocess_input as pr_152


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s:%(name)s:%(message)s')

file_handler = logging.FileHandler('prepare_images.log')
file_handler.setFormatter(formatter)

stream_handler = logging.StreamHandler()

logger.addHandler(file_handler)
logger.addHandler(stream_handler)


def read_image_paths(dir_path):
    """
    read the path of images in 'dir_path' and return a dictionary mapping image_id to image path.
    Arguments:
    dir_path -- a directory that consists of images.
    Return:
    ims -- a dictionary that maps image_id to image path.
    """
    ims = {}

    for filename in os.listdir(dir_path):
        if filename.endswith('.jpg'):
            image_id = int(re.findall('\d+', filename)[1])
            ims[image_id] = os.path.join(dir_path, filename)

    return ims


def load_and_proccess_image(image_path, model, image_size):
    """
    load and preprocess image, then extract features using model.
    Arguments:
    image_path -- a string that is image path.
    model -- a cnn model for extracting features from image.
    image_size -- expected size of image after loading.
    Return:
    features -- extracted features from image.
    """

    im = load_img(image_path, target_size=image_size)
    x = img_to_array(im)
    x = np.expand_dims(x, axis=0)
    x =  preprocess_input(x)

    features = model.predict(x)
    # x=np.squeeze(features[0])

    return features[0]


def extract_features(paths):
    """
    extract features from images using VGG19.
    Arguments:
    paths -- a dictionary that maps image_ids to image paths.
    Return:
    ims -- a dictionary that saves mapping between image_ids and image features.
    """
    ims = {}
    # base_model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')
    # fl = tensorflow.keras.layers.Flatten()(base_model.output)
    # model = Model(inputs=base_model.input,
    #               outputs=fl)
    base_model = ResNet152(weights='imagenet', include_top=False, pooling='avg')
    # fl = tensorflow.keras.layers.Flatten()(base_model.output)
    model = Model(inputs=base_model.input,
                  outputs=base_model.output)


    num_ims = len(paths)
    for i, (image_id, image_path) in enumerate(paths.items()):
        ims[image_id] = load_and_proccess_image(
            image_path, model, (331, 331))

        if (i+1) % 100 == 0:
            logger.info("extract features from %i/%i images." %
                        (i + 1, num_ims))
    return ims





def get_train_image_paths():
    return read_image_paths(IMAGE_TRAIN_PATH)


def get_val_image_paths():
    return read_image_paths(IMAGE_VAL_PATH)


def get_test_image_paths():
    return read_image_paths(IMAGE_TEST_PATH)


def save_train_features():
    """
    extract features from train images using VGG19 and save them as .csv and .h5 file.
    """
    logger.info("Start: extract features from train images.")
    train_ims = extract_features(get_train_image_paths())
    logger.info("End: extract features from train images.")

    # .csv
    # df = pd.DataFrame(train_ims)
    # df.to_csv('dataset/vgg19/X_train_ims_VGG19.csv')
    # logger.info('saved in \"X_train_ims_VGG19.csv\".')

    # .h5
    dd.io.save('train_full_features_inceptionresnet2_dict.h5',
               train_ims, compression=None)
    logger.info('saved in \"X_train_ims_VGG19.h5\".')


def save_val_features():
    """
    extract features from validation images using VGG19 and save them as .csv and .h5 file.
    """
    logger.info("Start: extract features from val images.")
    val_ims = extract_features(get_val_image_paths())
    logger.info("End: extract features from val images.")

    # # .csv
    # df = pd.DataFrame(val_ims)
    # df.to_csv('X_val_ims_VGG19.csv')
    # logger.info('saved in \"X_val_ims_VGG19.csv\".')

    # .h5
    dd.io.save('val_full_features_inceptionresnet2_dict.h5',
               val_ims, compression=None)
    logger.info('saved in \"X_val_ims_VGG19.h5\".')


def save_test_features():
    """
    extract features from test images using VGG19 and save them as .csv and .h5 file.
    """
    logger.info("Start: extract features from test images.")
    test_ims = extract_features(get_test_image_paths())
    logger.info("End: extract features from test images.")

    # .csv
    df = pd.DataFrame(test_ims)
    df.to_csv('X_test_ims_VGG19.csv')
    logger.info('saved in \"X_test_ims_VGG19.csv\."')

    # .h5
    dd.io.save('X_test_ims_VGG19.h5',
               test_ims, compression=None)
    logger.info('saved in \"X_test_ims_VGG19.h5\".')


save_train_features()
# save_val_features()
# save_test_features()

extract features from 65600/82783 images.
extract features from 65700/82783 images.
extract features from 65800/82783 images.
extract features from 65900/82783 images.
extract features from 66000/82783 images.
extract features from 66100/82783 images.
extract features from 66200/82783 images.
extract features from 66300/82783 images.
extract features from 66400/82783 images.
extract features from 66500/82783 images.
extract features from 66600/82783 images.
extract features from 66700/82783 images.
extract features from 66800/82783 images.
extract features from 66900/82783 images.
extract features from 67000/82783 images.
extract features from 67100/82783 images.
extract features from 67200/82783 images.
extract features from 67300/82783 images.
extract features from 67400/82783 images.
extract features from 67500/82783 images.
extract features from 67600/82783 images.
extract features from 67700/82783 images.
extract features from 67800/82783 images.
extract features from 67900/82783 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp /content/train_full_features_resnet152_dict.h5 /content/drive/My\ Drive/parssoftco_PVQA3/train_full_features_resnet152_dict.h5 

In [ ]:
!rm -rf /content/train2014/ #/content/drive/My Drive/parssoftco_PVQA3/val_full_features_VGG19_dict.h5

In [ ]:
def load_and_proccess_image2(image_path, model, image_size):
    """
    load and preprocess image, then extract features using model.
    Arguments:
    image_path -- a string that is image path.
    model -- a cnn model for extracting features from image.
    image_size -- expected size of image after loading.
    Return:
    features -- extracted features from image.
    """

    im = load_img(image_path, target_size=image_size)
    x = img_to_array(im)
    x = np.expand_dims(x, axis=0)
    x =  preprocess_input(x)

    features = model.predict(x)
    # x=np.squeeze(features[0])

    return features[0]


In [ ]:
import os
import re
import logging
import numpy as np
import pandas as pd
from PIL import Image
import deepdish as dd
# from constants import *
from tensorflow.keras.models import Model
# from tensorflow.keras.applications import VGG19
# from tensorflow.keras.applications import VGG16
# from tensorflow.keras.applications.vgg16 import preprocess_input as er
# from tensorflow.keras.applications.vgg19 import preprocess_input


from tensorflow.keras.preprocessing.image import load_img, img_to_array
def extract_features_one(image_path):
    """
    extract features from images using VGG19.
    Arguments:
    paths -- a dictionary that maps image_ids to image paths.
    Return:
    ims -- a dictionary that saves mapping between image_ids and image features.
    """
    ims = {}
    base_model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')
    fl = tensorflow.keras.layers.Flatten()(base_model.output)
    model = Model(inputs=base_model.input,
                  outputs=fl)

    
    
    # base_model = VGG16(weights='imagenet', include_top=True)
    # model = Model(inputs=base_model.input,
    #               outputs=base_model.get_layer('fc2').output)

    image_paths = read_image_paths(IMAGE_TRAIN_PATH)

    
    ims= load_and_proccess_image2(image_paths[487025], model, (229, 229))


    return ims

In [ ]:
q=extract_features_one('/content/train2014/')

In [ ]:
q.shape

(1536,)

In [ ]:
a

array([2.138524 , 6.1470633, 0.       , ..., 0.       , 0.       ,
       0.       ], dtype=float32)

In [ ]:
q==a

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
import deepdish as dd

In [ ]:
b

array([0.       , 1.0902787, 0.       , ..., 0.       , 0.       ,
       0.       ], dtype=float32)

In [ ]:
dict_old_us=dd.io.load('/content/drive/My Drive/parssoftco_PVQA3/train_imid_to_feats_dict.h5')

In [ ]:
dict=dd.io.load('/content/drive/My Drive/parssoftco_PVQA3/train_full_features_VGG19_dict.h5')

In [ ]:
len(dict)

82783

In [ ]:
a=dict[487025]

In [ ]:
b=dict_old_us[487025]

In [ ]:
a==b

array([False, False,  True, ...,  True,  True,  True])

In [ ]:
!pip install --upgrade numpy
!pip install --upgrade tables

     |████████████████████████████████| 14.5MB 227kB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [ ]:
dict[]